In [27]:
import numpy as np
import pandas as pd
from keras import models
from keras import layers
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification

In [28]:
data = pd.read_csv('Data.csv')

In [29]:
data['normalizedAmount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1,1))
data.drop(['Amount', 'Time'], axis=1, inplace=True)
train = data.drop(['Class'], axis=1)
label = data['Class']

In [30]:
train_set, test_set, train_label, test_label = train_test_split(train, label, test_size=0.2)


In [31]:
def build_classifier(units=16, learn_rate=0.001, hidden_layers=1):
    m =  models.Sequential()

    m.add(layers.Dense(units=29, activation='relu', input_shape=(29,)))
    for i in range(hidden_layers):
        m.add(layers.Dense(units=units, activation='relu'))
    m.add(layers.Dense(units=1, activation='sigmoid'))
    optimizer = optimizers.Adam(learning_rate=learn_rate)
    m.compile(optimizer=optimizer, loss='binary_crossentropy',
             metrics=['accuracy'])
    return m

In [32]:
classifier = KerasClassifier(build_fn=build_classifier)

In [33]:
parameters ={
            'epochs':[10,20,30],
            'units':[16, 32, 48, 64],
            'batch_size': [16, 32, 64],
            'learn_rate': [0.001, 0.003, 0.01],
            'hidden_layers': [1, 2, 3]}
gridSearch = GridSearchCV(estimator=classifier,
                          param_grid=parameters,
                         n_jobs=-1)

In [34]:
grid_result = gridSearch.fit(train, label, verbose=1)

/Users/prial/opt/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/20
1400/1400 [==============================] - 1s 689us/step - loss: 0.2323 - accuracy: 0.9271
Epoch 2/20
1400/1400 [==============================] - 0s 66us/step - loss: 0.1125 - accuracy: 0.9579
Epoch 3/20
1400/1400 [==============================] - 0s 64us/step - loss: 0.0908 - accuracy: 0.9693
Epoch 4/20
1400/1400 [==============================] - 0s 64us/step - loss: 0.0766 - accuracy: 0.9729
Epoch 5/20
1400/1400 [==============================] - 0s 64us/step - loss: 0.0753 - accuracy: 0.9779
Epoch 6/20
1400/1400 [==============================] - 0s 64us/step - loss: 0.0576 - accuracy: 0.9764
Epoch 7/20
1400/1400 [==============================] - 0s 67us/step - loss: 0.0551 - accuracy: 0.9807
Epoch 8/20
1400/1400 [==============================] - 0s 65us/step - loss: 0.0462 - accuracy: 0.9800
Epoch 9/20
1400/1400 [==============================] - 0s 64us/step - loss: 0.0436 - accuracy: 0.9843
Epoch 10/20
1400/1400 [==============================] - 0s 64us/step - 

In [35]:
print("Best: %f using %s"%(grid_result.best_score_,grid_result.best_params_))
means=grid_result.cv_results_['mean_test_score']
stds=grid_result.cv_results_['std_test_score']
params=grid_result.cv_results_['params']
for mean,stdev,param in zip(means,stds,params):
    print("%f (%f) with: %r"%(mean,stdev,param))

Best: 0.962857 using {'batch_size': 32, 'epochs': 20, 'hidden_layers': 3, 'learn_rate': 0.01, 'units': 16}
0.957143 (0.003498) with: {'batch_size': 16, 'epochs': 10, 'hidden_layers': 1, 'learn_rate': 0.001, 'units': 16}
0.956429 (0.005379) with: {'batch_size': 16, 'epochs': 10, 'hidden_layers': 1, 'learn_rate': 0.001, 'units': 32}
0.945714 (0.007115) with: {'batch_size': 16, 'epochs': 10, 'hidden_layers': 1, 'learn_rate': 0.001, 'units': 48}
0.960714 (0.005376) with: {'batch_size': 16, 'epochs': 10, 'hidden_layers': 1, 'learn_rate': 0.001, 'units': 64}
0.960000 (0.002680) with: {'batch_size': 16, 'epochs': 10, 'hidden_layers': 1, 'learn_rate': 0.003, 'units': 16}
0.956429 (0.008280) with: {'batch_size': 16, 'epochs': 10, 'hidden_layers': 1, 'learn_rate': 0.003, 'units': 32}
0.957857 (0.000968) with: {'batch_size': 16, 'epochs': 10, 'hidden_layers': 1, 'learn_rate': 0.003, 'units': 48}
0.957143 (0.004642) with: {'batch_size': 16, 'epochs': 10, 'hidden_layers': 1, 'learn_rate': 0.003, 'u